# Загрузка библиотек

In [1]:
import numpy as np
import scipy.stats as stats
import scipy.special as special

# Тест Пирсона

In [2]:
import numpy as np
from scipy import special, stats

def forpirson(x, n, k):
    """
    Функция для вычисления весов и вероятностей в тесте Пирсона.
    
    Параметры:
    x (numpy.ndarray): Входные данные.
    n (int): Количество наблюдений.
    k (int): Количество интервалов.
    
    Возвращает:
    w (numpy.ndarray): Веса для каждого интервала.
    p (numpy.ndarray): Вероятности для каждого интервала.
    """
    z0 = np.floor(min(x))
    zk = np.ceil(max(x))
    h = (zk - z0) / k
    w = np.zeros(k)
    i = 0
    for _x in x:
        z0 -= i * h
        i = 0
        while z0 < zk:
            if z0 <= _x <= z0 + h:
                w[i] += 1
            i += 1
            z0 += h
    z0 -= i * h
    w /= n
    z = []
    for i in range(k):
        z.append((z0 + h * i + z0 + h * (i + 1)) / 2)
    z = np.array(z)
    xavg = (z * w).sum()
    xvar = (pow((z - xavg), 2) * w).sum() * k / (k - 1)
    p = []
    for i in range(k):
        p.append((special.erf((z0 + h * (i + 1) - xavg) / np.sqrt(2 * xvar)) - special.erf((z0 + h * (i) - xavg) / np.sqrt(2 * xvar))) / 2)
    p = np.array(p)
    return w, p

def pirson(p1, p2, n):
    """
    Функция для вычисления статистики теста Пирсона.
    
    Параметры:
    p1 (numpy.ndarray): Фактические вероятности.
    p2 (numpy.ndarray): Теоретические вероятности.
    n (int): Количество наблюдений.
    
    Возвращает:
    статистику теста Пирсона.
    """
    if min(p2) > 0:
        return n * (pow((p1 - p2), 2) / p2).sum()
    else:
        return np.inf

In [3]:
n = 1000
alpha = 0.05
k = int(np.floor(np.log2(n)) + 1)

In [4]:
k

10

In [6]:
# Нормальное распределение
x = np.random.normal(3, np.sqrt(5), n)
w, p = forpirson(x, n, k)
ans = pirson(w, p, n)
chi2 = stats.chi2.ppf(1 - alpha, k - 1)
print("Нормальное распределение:")
print("=" * 50)
print(f"Статистика Пирсона: {ans:.2f}")
print(f"Критическое значение χ²: {chi2:.2f}")
print("=" * 50)
print(f"Гипотеза о нормальном распределении {'отвергается' if ans >= chi2 else 'принимается'}\n")

Нормальное распределение:
Статистика Пирсона: 10.36
Критическое значение χ²: 16.92
Гипотеза о нормальном распределении принимается



In [51]:
# Распределение Коши
y = x + 0.5 * np.random.standard_cauchy(n)
w, p = forpirson(y, n, k)
ans = pirson(w, p, n)
print("Распределение Коши:")
print("=" * 50)
print(f"Статистика Пирсона: {ans:.2f}")
print(f"Критическое значение χ²: {chi2:.2f}")
print("=" * 50)
print(f"Гипотеза о распределении Коши {'отвергается' if ans >= chi2 else 'принимается'}\n")

Распределение Коши:
Статистика Пирсона: inf
Критическое значение χ²: 16.92
Гипотеза о распределении Коши отвергается



In [7]:
# Равномерное распределение
z = x + 0.3 * np.random.uniform(-1, 1, n)
w, p = forpirson(z, n, k)
ans = pirson(w, p, n)
print("Равномерное распределение:")
print("=" * 50)
print(f"Статистика Пирсона: {ans:.2f}")
print(f"Критическое значение χ²: {chi2:.2f}")
print("=" * 50)
print(f"Гипотеза о гауссовском распределении {'отвергается' if ans >= chi2 else 'принимается'}")

Равномерное распределение:
Статистика Пирсона: 11.00
Критическое значение χ²: 16.92
Гипотеза о гауссовском распределении принимается


# Тест Колмогорова

In [8]:
def forkolmogorov(x, xj):
    """
    Функция для вычисления количества элементов в выборке, меньших или равных xj.
    
    Параметры:
    x (numpy.ndarray): Входные данные.
    xj (float): Значение, для которого вычисляется количество элементов.
    
    Возвращает:
    int: Количество элементов в выборке, меньших или равных xj.
    """
    sum = 0
    for _x in x:
        if _x <= xj:
            sum += 1
    return sum

def kolmogorov(x, n, eps=0.0001):
    """
    Функция для вычисления статистики Колмогорова-Смирнова.
    
    Параметры:
    x (numpy.ndarray): Входные данные.
    n (int): Количество наблюдений.
    eps (float, optional): Малая величина для вычисления Fn_. По умолчанию 0.0001.
    
    Возвращает:
    float: Статистика Колмогорова-Смирнова.
    """
    D = 0
    for _x in x:
        F = special.erf((_x - np.average(x)) / np.sqrt(2 * np.var(x))) / 2 + 0.5
        Fn__ = forkolmogorov(x, _x + eps) / n
        Fn_ = forkolmogorov(x, _x - eps) / n
        D = max(abs(F - Fn_), abs(F - Fn__), D)
    return np.sqrt(n) * D

In [9]:
# Нормальное распределение
x = np.random.normal(3, np.sqrt(5), n)
ans = kolmogorov(x, n)
kolm = stats.kstwobign.ppf(1 - alpha)
print("Нормальное распределение:")
print("=" * 50)
print(f"Статистика Колмогорова-Смирнова: {ans:.4f}")
print(f"Критическое значение: {kolm:.4f}")
print("=" * 50)
print(f"Гипотеза о нормальном распределении {'отвергается' if ans >= kolm else 'принимается'}\n")

Нормальное распределение:
Статистика Колмогорова-Смирнова: 0.5993
Критическое значение: 1.3581
Гипотеза о нормальном распределении принимается



In [10]:
# Распределение Коши
y = x + 0.5 * np.random.standard_cauchy(n)
ans = kolmogorov(y, n)
print("Распределение Коши:")
print("=" * 50)
print(f"Статистика Колмогорова-Смирнова: {ans:.4f}")
print(f"Критическое значение: {kolm:.4f}")
print("=" * 50)
print(f"Гипотеза о распределении Гаусса {'отвергается' if ans >= kolm else 'принимается'}\n")

Распределение Коши:
Статистика Колмогорова-Смирнова: 8.2783
Критическое значение: 1.3581
Гипотеза о распределении Гаусса отвергается



In [11]:
# Равномерное распределение
z = x + 0.3 * np.random.uniform(-1, 1, n)
ans = kolmogorov(z, n)
print("Равномерное распределение:")
print("=" * 50)
print(f"Статистика Колмогорова-Смирнова: {ans:.4f}")
print(f"Критическое значение: {kolm:.4f}")
print("=" * 50)
print(f"Гипотеза о Гауссова распределении {'отвергается' if ans >= kolm else 'принимается'}")

Равномерное распределение:
Статистика Колмогорова-Смирнова: 0.4848
Критическое значение: 1.3581
Гипотеза о Гауссова распределении принимается


# Двухвыборочный тест Колмогорова-Смирнова

In [12]:
def forkolmogorov(x, xj):
    """
    Функция для вычисления количества элементов в выборке, меньших или равных xj.
    
    Параметры:
    x (numpy.ndarray): Входные данные.
    xj (float): Значение, для которого вычисляется количество элементов.
    
    Возвращает:
    int: Количество элементов в выборке, меньших или равных xj.
    """
    sum = 0
    for _x in x:
        if _x <= xj:
            sum += 1
    return sum

def kolmogorovsmirnov(x1, x2, n, eps = 0.0001):
    D = 0
    for _x in x1:
        F11_ = forkolmogorov(x1, _x - eps) / n
        F11__ = forkolmogorov(x1, _x + eps) / n
        F21 = forkolmogorov(x2, _x) / n
        for __x in x2:
            F12 = forkolmogorov(x1, __x) / n
            F22_ = forkolmogorov(x2, __x - eps) / n
            F22__ = forkolmogorov(x2, __x + eps) / n
            D = max(abs(F11_ - F21), abs(F11__ - F21), abs(F12 - F22_), abs(F12 - F22__), D)
    return np.sqrt(n / 2) * D

In [56]:
# Тест Колмогорова-Смирнова для двух выборок
x = np.random.normal(3, np.sqrt(5), n)
ans = kolmogorovsmirnov(x[:500], x[500:], 500)
print("Тест Колмогорова-Смирнова для двух выборок (одно и то же распределение):")
print("=" * 50)
print(f"Статистика: {ans:.4f}")
print(f"Критическое значение: {kolm:.4f}")
print("=" * 50)
print(f"Гипотеза о равенстве распределений {'отвергается' if ans >= kolm else 'принимается'}\n")

Тест Колмогорова-Смирнова для двух выборок (одно и то же распределение):
Статистика: 0.4427
Критическое значение: 1.3581
Гипотеза о равенстве распределений принимается



In [13]:
x = np.random.normal(3, np.sqrt(5), n)
x2 = np.random.normal(3, np.sqrt(6), n)
ans = kolmogorovsmirnov(x[:500], x2[:500], 500)
print("Тест Колмогорова-Смирнова для двух выборок (разные распределения):")
print("=" * 50)
print(f"Статистика: {ans:.4f}")
print(f"Критическое значение: {kolm:.4f}")
print("=" * 50)
print(f"Гипотеза о равенстве распределений {'отвергается' if ans >= kolm else 'принимается'}")

Тест Колмогорова-Смирнова для двух выборок (разные распределения):
Статистика: 1.1700
Критическое значение: 1.3581
Гипотеза о равенстве распределений принимается
